In [2]:
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

dftrain=pd.read_csv("/content/RavdessAudioOnlyFeatures_TRAIN.csv")
dftest=pd.read_csv("/content/RavdessAudioOnlyFeatures_TEST.csv")
del dftrain["filename"]
del dftest["filename"]
del dftrain["modality"]
del dftest["modality"]

from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
dftrain.iloc[:,7:]=pd.DataFrame(scaler.fit_transform(dftrain.iloc[:,7:]))
dftest.iloc[:,7:]=pd.DataFrame(scaler.fit_transform(dftest.iloc[:,7:]))
y_train=dftrain.iloc[:,0]
y_test=dftest.iloc[:,0]
X_train=dftrain.iloc[:,7:]
X_test=dftest.iloc[:,7:]

from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier

sel = RFE(DecisionTreeClassifier(), n_features_to_select=9)
X_train_sel = sel.fit_transform(X_train, y_train)
X_test_sel = sel.transform(X_test)
selected_indices = sel.get_support(indices=True)
selected_feature_names = X_train.columns[selected_indices]
X_train = pd.DataFrame(X_train_sel, columns=selected_feature_names)
X_test = pd.DataFrame(X_test_sel, columns=selected_feature_names)

def replace_outlier(val, mean, std):
    if val > mean + 2.57*std:
        return mean
    elif val < mean - 2.57*std:
        return mean
    return val

for col in X_train.iloc[:,0:len(X_train)]:
    mean = X_train[col].mean()
    std_dev = X_train[col].std(axis=0)
    X_train[col] = X_train[col].map(lambda x: replace_outlier(x, mean, std_dev))

y_train=pd.DataFrame(dftrain.iloc[:,0])
y_test=pd.DataFrame(dftest.iloc[:,0])

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_train_dummy = pd.DataFrame(label_encoder.fit_transform(y_train))
y_test_dummy = pd.DataFrame(label_encoder.fit_transform(y_test))



<ipython-input-2-675bf0a7bb6d>:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dftrain.iloc[:,7:]=pd.DataFrame(scaler.fit_transform(dftrain.iloc[:,7:]))
<ipython-input-2-675bf0a7bb6d>:18: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  dftest.iloc[:,7:]=pd.DataFrame(scaler.fit_transform(dftest.iloc[:,7:]))
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y =

In [6]:
df = pd.concat([X_train, X_test])

In [7]:
class_name = 'vocal_channel'
attributes = [c for c in df.columns if c != class_name]

In [11]:
!pip install datamanager

In [15]:
from collections import Counter
from collections import defaultdict


from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score

scaler = MinMaxScaler()
X_tr_sc = scaler.fit_transform(X_train)
df_tr_sc = pd.DataFrame(X_tr_sc)
X_te_sc = scaler.fit_transform(X_test)
df_te_sc = pd.DataFrame(X_te_sc)
X_sc = scaler.fit_transform(df)
df_sc = pd.DataFrame(X_sc)



CLASSIFICATION

In [19]:
from sklearn.ensemble import RandomForestClassifier

bb = RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, min_samples_split=2,
                             min_samples_leaf=3, min_weight_fraction_leaf=0.0, max_features='auto', random_state=0)
bb.fit(X_train, y_train)

y_pred = bb.predict(X_test)

def bb_predict(X):
    return bb.predict(X)

def bb_predict_proba(X):
    return bb.predict_proba(X)
y_pred = bb_predict(X_test)

print('Accuracy %.3f' % accuracy_score(y_test, y_pred))
from sklearn.metrics import f1_score

f1_measure_song = f1_score(y_test, y_pred, pos_label='song')
print('F1-measure for song %.3f' % f1_measure_song)

f1_measure_speech = f1_score(y_test, y_pred, pos_label='speech')
print('F1-measure for speech %.3f' % f1_measure_speech)

<ipython-input-19-865078fa83e7>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  bb.fit(X_train, y_train)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Accuracy 0.960
F1-measure for song 0.953
F1-measure for speech 0.965


EXPLANATION

In [28]:
from skater.model import InMemoryModel
from skater.core.explanations import Interpretation

from skater.core.global_interpretation.partial_dependence import PartialDependence

interpreter = Interpretation()
interpreter.load_data(X_train, feature_names=attributes)

model = InMemoryModel(bb_predict_proba, examples=X_test)
interpreter.partial_dependence.plot_partial_dependence([attributes[0], attributes[1]],
                                                       model, n_samples=100, n_jobs=1)
plt.show()

ModuleNotFoundError: ignored

In [ ]:
model = InMemoryModel(bb_predict_proba, examples=X_test)
interpreter.partial_dependence.plot_partial_dependence([attributes[2], attributes[3]],
                                                       model, n_samples=100, n_jobs=1)
plt.show()

LOCAL EXPLAINATION
select a record to explain

In [40]:
X_test.iloc[22,:]

q25            0.695268
q99           -0.822569
kur            0.598215
sc_q25        -0.690463
sc_q75_w2      0.866461
sc_skew_w2    -0.437943
length_w4     -1.727157
skew_w4        1.125884
stft_min_w4   -0.296250
Name: 22, dtype: float64

In [38]:
!pip install util

ERROR: Could not find a version that satisfies the requirement util (from versions: none)
ERROR: No matching distribution found for util


In [49]:
bb_outcome

'speech'

In [48]:

from random import seed
from random import randint
# seed random number generator
seed(1)
# generate some integers
i2e = randint(0, len(X_test)-1)

x = X_test.iloc[i2e,:]

class_values = [0, 1]

bb_outcome = bb_predict(np.array(x).reshape(1, -1))[0]
bb_outcome_str = class_values[bb_outcome]

print('bb(x) = {}' % bb_outcome_str)
print('')


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


TypeError: ignored

LIME

In [51]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283839 sha256=a36896953f8846da0a0046f866ed8d766d7640e0bf9c3e4db8530c0dd7d6f50f
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


In [52]:
from lime.lime_tabular import LimeTabularExplainer
lime_explainer = LimeTabularExplainer(X_test, feature_names=attributes,
                                      class_names=['0','1'], discretize_continuous=False)
exp = lime_explainer.explain_instance(x, bb_predict_proba)
exp.local_exp

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


{1: [(6, -0.14948760846454492),
  (4, 0.10334394956462796),
  (5, -0.06962615549572747),
  (3, 0.05698922086389098),
  (8, 0.05659081120762901),
  (0, 0.050352249080322535),
  (2, 0.047545138067605575),
  (1, -0.014633382035843804),
  (7, 0.0042189509872026885)]}

In [53]:
exp.show_in_notebook()

SHAP

In [54]:
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 9.2 MB/s eta 0:00:00


In [55]:
import shap

shap.initjs()

In [68]:
f = lambda x: bb_predict_proba(x)[:, 1]
med = np.median(X_test, axis=0).reshape((1, X_test.shape[1]))

shap_explainer = shap.KernelExplainer(f, med)

shap_values_single = shap_explainer.shap_values(x, nsamples=1000)
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values_single, features=x, feature_names=attributes)

X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names


In [69]:
shap_values = shap_explainer.shap_values(X_test, nsamples=1000)
shap.initjs()
shap.force_plot(shap_explainer.expected_value, shap_values, X_test, feature_names=attributes)

  0%|          | 0/624 [00:00<?, ?it/s]

X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassifier was fitted with feature names
X does not have valid feature names, but RandomForestClassif

LORE

In [60]:
!pip install lorem

In [63]:
!pip install --upgrade lorem

In [67]:
from lorem import LOREM

def get_features_map(feature_names, real_feature_names):
    features_map = defaultdict(dict)
    i = 0
    j = 0

    while i < len(feature_names) and j < len(real_feature_names):
        if feature_names[i] == real_feature_names[j]:
            features_map[j][feature_names[i]] = j
            i += 1
            j += 1
        elif feature_names[i].startswith(real_feature_names[j]):
            features_map[j][feature_names[i]] = j
            i += 1
        else:
            j += 1
    return features_map
features_map = get_features_map(attributes, attributes)
lore_explainer = LOREM(X_test, bb_predict, attributes, class_name, class_values, attributes, features_map,
                       neigh_type='geneticp', categorical_use_prob=True, continuous_fun_estimation=False,
                       size=1000, ocr=0.1, random_state=42, ngen=10, bb_predict_proba=bb_predict_proba,
                       verbose=True)

ImportError: ignored

In [ ]:
exp = lore_explainer.explain_instance(x, samples=1000, use_weights=True, metric=neuclidean)

In [ ]:
print(exp)

In [ ]:
x1 = x.copy()
x1[attributes.index('Light')] = 290

print('x = %s' % record2str(x, attributes, attributes))
print('')
print('x = %s' % record2str(x1, attributes, attributes))
print('')

In [ ]:
bb_outcome = bb_predict(x1.reshape(1, -1))[0]
bb_outcome_str = class_values[bb_outcome]

print('bb(x) = { %s }' % bb_outcome_str)
print('')